## **Epochs Testing**  

In [1]:

import pandas as pd
import os
import time
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

vector_size_num = 100
window_num = 5
workers_num = 4
negative_sampling = 10
epoches_num = 10
min_count = 2
w2v_min_count = 2
w2v_topn = 10

class LossCallback(CallbackAny2Vec):
 
    def __init__(self):
        self.epoch = 0
        self.previous_loss = 0
        self.loss_history = []
        
    def on_epoch_end(self, model):
        current_loss = model.get_latest_training_loss()
        
       
        if self.epoch == 0:
            loss = current_loss
        else:
            loss = current_loss - self.previous_loss
            
        self.loss_history.append(loss)
        self.previous_loss = current_loss
        print(f'Epoch {self.epoch}, Loss: {loss}')
        self.epoch += 1

def load_transaction_data(file_path):
 
    print(f"Loading data from {file_path}")

    data = pd.read_excel(file_path)
    data.dropna(subset=['Itemname'], inplace=True)
    
   
    basket = data.groupby('BillNo')['Itemname'].apply(list)
    transactions = basket.values.tolist()
    
    print(f"Loaded {len(transactions)} transactions")
    return transactions

def train_model_with_epochs(transactions, epochs, vector_size=100, window=5, min_count=2, workers=4, negative=10):
  
    print(f"Training model with {epochs} epochs...")
    

    loss_callback = LossCallback()
    

    start_time = time.time()
    model = Word2Vec(
        sentences=transactions,
        vector_size=vector_size,
        window=window,
        sg=1,  
        negative=negative,
        min_count=min_count,
        workers=workers,
        epochs=epochs,
        compute_loss=True,
        callbacks=[loss_callback]
    )
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    
    return {
        'epochs': epochs,
        'training_time': training_time,
        'loss_history': loss_callback.loss_history,
        'final_loss': loss_callback.loss_history[-1] if loss_callback.loss_history else None
    }

def test_multiple_epochs(transactions, epoch_values):

    results = {}
    
    for epochs in epoch_values:
        print(f"\n===== Testing {epochs} epochs =====")
        result = train_model_with_epochs(transactions, epochs)
        results[epochs] = result
    
    return results

def calculate_convergence(results):
 
    epochs = sorted(list(results.keys()))
    final_losses = [results[e]['final_loss'] for e in epochs]
    
   
    loss_decreases = []
    for i in range(1, len(epochs)):
        if final_losses[i-1] > 0 and final_losses[i] > 0:
            percentage_decrease = (final_losses[i-1] - final_losses[i]) / final_losses[i-1] * 100
            loss_decreases.append((epochs[i], percentage_decrease))
  
    print("\nLoss decrease between epochs:")
    for epoch, decrease in loss_decreases:
        print(f"From {epochs[epochs.index(epoch)-1]} to {epoch} epochs: {decrease:.2f}% decrease")
    

    threshold = 5.0  
    for epoch, decrease in loss_decreases:
        if decrease < threshold:
            print(f"\nDiminishing returns detected at {epoch} epochs (only {decrease:.2f}% improvement)")
            recommended_epochs = epoch
            break
    else:
   
        recommended_epochs = epochs[-1]
        print(f"\nNo clear diminishing returns detected. Consider testing with more epochs.")
    

    print("\nEpoch testing summary:")
    summary_data = {
        'Epochs': epochs,
        'Final Loss': [round(loss, 4) for loss in final_losses],
        'Training Time (s)': [round(results[e]['training_time'], 2) for e in epochs]
    }
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    print(f"\nRecommended number of epochs based on convergence: {recommended_epochs}")
    return recommended_epochs

def main():

    data_path = r'C:\Users\moham\Apriori_VS_Word2Vec\Dataset'
    excel_file = 'df_merged_items_category.xlsx'
    file_path = os.path.join(data_path, excel_file)

    epoch_values = [5, 10, 15, 20, 25, 30]

    transactions = load_transaction_data(file_path)

    results = test_multiple_epochs(transactions, epoch_values)
    
    optimal_epochs = calculate_convergence(results)

    print(f"\nTo use this value in your original code, replace:")
    print("epoches_num = 10  # Current value")
    print(f"with:")
    print(f"epoches_num = {optimal_epochs}  # Optimized value based on convergence")

if __name__ == "__main__":
    main()
    


Loading data from C:\Users\moham\Apriori_VS_Word2Vec\Dataset\df_merged_items_category.xlsx


2025-04-15 21:49:30,914 : INFO : collecting all words and their counts
2025-04-15 21:49:30,915 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-04-15 21:49:30,927 : INFO : PROGRESS: at sentence #10000, processed 244376 words, keeping 3614 word types
2025-04-15 21:49:30,944 : INFO : PROGRESS: at sentence #20000, processed 513626 words, keeping 4179 word types
2025-04-15 21:49:30,944 : INFO : collected 4185 word types from a corpus of 520609 raw words and 20208 sentences
2025-04-15 21:49:30,945 : INFO : Creating a fresh vocabulary
2025-04-15 21:49:30,953 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 3877 unique words (92.64% of original 4185, drops 308)', 'datetime': '2025-04-15T21:49:30.953856', 'gensim': '4.3.3', 'python': '3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'prepare_vocab'}
2025-04-15 21:49:30,953 : INFO : Word2Vec 

Loaded 20208 transactions

===== Testing 5 epochs =====
Training model with 5 epochs...


2025-04-15 21:49:31,748 : INFO : EPOCH 0: training on 520609 raw words (518229 effective words) took 0.8s, 686587 effective words/s


Epoch 0, Loss: 2030243.375


...

Epoch 27, Loss: 1040336.0


2025-04-15 21:51:01,087 : INFO : EPOCH 28: training on 520609 raw words (518217 effective words) took 0.8s, 612898 effective words/s


Epoch 28, Loss: 1110132.0


2025-04-15 21:51:01,943 : INFO : EPOCH 29: training on 520609 raw words (518195 effective words) took 0.9s, 608903 effective words/s
2025-04-15 21:51:01,943 : INFO : Word2Vec lifecycle event {'msg': 'training on 15618270 raw words (15547904 effective words) took 26.6s, 584708 effective words/s', 'datetime': '2025-04-15T21:51:01.943086', 'gensim': '4.3.3', 'python': '3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'train'}
2025-04-15 21:51:01,944 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=3877, vector_size=100, alpha=0.025>', 'datetime': '2025-04-15T21:51:01.944176', 'gensim': '4.3.3', 'python': '3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'created'}


Epoch 29, Loss: 1054848.0
Training completed in 26.68 seconds

Loss decrease between epochs:
From 5 to 10 epochs: 8.14% decrease
From 10 to 15 epochs: 6.28% decrease
From 15 to 20 epochs: 3.16% decrease
From 20 to 25 epochs: 0.26% decrease
From 25 to 30 epochs: 11.89% decrease

Diminishing returns detected at 20 epochs (only 3.16% improvement)

Epoch testing summary:
 Epochs  Final Loss  Training Time (s)
      5   1439677.5               4.07
     10   1322545.0               8.33
     15   1239458.0              12.76
     20   1200246.0              17.29
     25   1197142.0              21.90
     30   1054848.0              26.68

Recommended number of epochs based on convergence: 20

To use this value in your original code, replace:
epoches_num = 10  # Current value
with:
epoches_num = 20  # Optimized value based on convergence
